In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pickle
import datetime
import tensorboard

In [45]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
data = data.drop(["RowNumber","CustomerId","Surname"], axis=1)

In [47]:
label_encoder_gender = LabelEncoder()

data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

In [48]:
Onehot_encode_geo = OneHotEncoder(handle_unknown="ignore")

geo_encoded = Onehot_encode_geo.fit_transform(data[["Geography"]]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns= Onehot_encode_geo.get_feature_names_out(["Geography"]))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [49]:
data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [50]:
X = data.drop("EstimatedSalary", axis=1)
y = data["EstimatedSalary"]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.2,random_state=42)

In [52]:
scaler = StandardScaler()

scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[-0.59170945,  0.89091075, -0.64371665, ..., -0.99600797,
         1.74133801, -0.58350885],
       [-0.31104689,  0.89091075,  0.40826421, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-0.5397349 , -1.12244688,  0.50389883, ..., -0.99600797,
        -0.57427105,  1.71377008],
       ...,
       [ 0.63488991,  0.89091075, -0.54808203, ..., -0.99600797,
        -0.57427105,  1.71377008],
       [ 0.47896626,  0.89091075, -0.35681278, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-1.39211751,  0.89091075, -0.45244741, ...,  1.00400803,
        -0.57427105, -0.58350885]])

In [53]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('one_hot_encoder.pkl','wb') as file:
    pickle.dump(Onehot_encode_geo, file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [54]:
## ANN WIth regression problem statement 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [55]:
model = Sequential([
    Dense(64, activation= "relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation= "relu"),
    Dense(1)
    
    
]
)

model.compile(optimizer ='adam',loss = 'mean_absolute_error',metrics=['mae'])

model.summary()

e:\Learning\Computer vision\data\ANN_classification_Churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq =1)

In [57]:
EarlyStopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [58]:
history = model.fit(
    X_train, y_train, 
    validation_data = (X_test,y_test),
    epochs= 100,
    callbacks = [EarlyStopping_callback, tensorboard_callback]
)
                

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 94594.9609 - mae: 94594.9609 - val_loss: 70111.0547 - val_mae: 70111.0547
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 69547.7188 - mae: 69547.7188 - val_loss: 68967.9688 - val_mae: 68967.9688
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 69915.7109 - mae: 69915.7109 - val_loss: 68685.6328 - val_mae: 68685.6328
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 70147.3359 - mae: 70147.3359 - val_loss: 68356.4609 - val_mae: 68356.4609
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 68532.6406 - mae: 68532.6406 - val_loss: 67815.5234 - val_mae: 67815.5234
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 69321.3906 - mae: 69321.3906 - val_loss: 66956.9531 - val_mae: 66956.9531
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 66175.1797 - mae: 66175.1797 - val_loss: 66072.4844 - val_mae: 66072.4844
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 66

In [59]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [64]:


%tensorboard --logdir regressionlogs/fit20250629-162132/

ERROR: Failed to launch TensorBoard (exited with 1).

In [65]:
#evaluate model

test_loss , test_mae = model.evaluate(X_test,y_test)
print(f"test MAE {test_mae}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 50470.0938 - mae: 50470.0938
test MAE 50687.28515625


In [66]:
model.save("Regression_model.h5")